In [1]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, confusion_matrix, classification_report

In [2]:
df = pd.read_csv('/home/jensk/code/JensKlug/zillows_real_estate/raw_data/HouseTS.csv')

# Make sure it's sorted by city, zipcode, and date
df = df.sort_values(['city', 'zipcode', 'date']).reset_index(drop=True)

# Convert date to datetime if not already
df['date'] = pd.to_datetime(df['date'])

train = df[df.date.dt.year<2023]
#train.shape

test = df[df.date.dt.year>2022]
#test.shape

In [3]:
#train.info()
#train.columns
df.head()

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,...,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
1,2012-04-30,61870.0,245000.0,40.723982,130.528256,22.0,29.0,56.0,69.0,89.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202421.064584,Atlanta-Sandy Springs-Alpharetta
2,2012-05-31,125500.0,217450.0,63.913043,119.919216,24.0,40.0,63.0,60.0,144.5,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202681.309539,Atlanta-Sandy Springs-Alpharetta
3,2012-06-30,153000.0,189900.0,81.598080,105.617353,34.0,46.0,50.0,57.0,126.0,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202998.603897,Atlanta-Sandy Springs-Alpharetta
4,2012-07-31,165500.0,154000.0,81.598080,83.921175,39.0,49.0,42.0,50.0,80.0,...,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,203781.903446,Atlanta-Sandy Springs-Alpharetta


In [4]:
# Group by city and zipcode for proper time series handling
grouped_train = train.groupby(['city', 'zipcode'])
grouped_test = test.groupby(['city', 'zipcode'])
#type(grouped_train), type(train)

In [5]:
grouped_train['price'].shift(-3), grouped_train['price'].shift(0)#, grouped_train['price'].shift(0).shape
#grouped_train['price']

(0         202998.603897
 1         203781.903446
 2         205012.252913
 3         205414.599202
 4         205196.396434
               ...      
 884075    427285.207121
 884076    424295.786194
 884077              NaN
 884078              NaN
 884079              NaN
 Name: price, Length: 809380, dtype: float64,
 0         200773.999557
 1         202421.064584
 2         202681.309539
 3         202998.603897
 4         203781.903446
               ...      
 884075    432523.250209
 884076    431488.991172
 884077    429747.893176
 884078    427285.207121
 884079    424295.786194
 Name: price, Length: 809380, dtype: float64)

In [6]:
_ = (grouped_train['price'].shift(-3) > 1.02 * grouped_train['price'].shift(0)).astype(int)
_.value_counts(), _

(price
 0    483916
 1    325464
 Name: count, dtype: int64,
 0         0
 1         0
 2         0
 3         0
 4         0
          ..
 884075    0
 884076    0
 884077    0
 884078    0
 884079    0
 Name: price, Length: 809380, dtype: int64)

In [7]:
#grouped_train_train.head(1) #With this command I get 6226 lines back which are the unique combinations of cities to zipcodes.
#train.head(1) #Here in contrast I just get one line back since it's a normal dataframe and not a groupby pandas object.

In [8]:
ts_train = pd.DataFrame()
# Create 3-month profitability target
#ts_train['target_profitable_3m'] = (grouped_train['price'].shift(-3) > 1.02 * train['price']).astype(int)
ts_train['target_profitable_3m'] = (grouped_train['price'].shift(-3) > 1.02 * grouped_train['price'].shift(0)).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!

numeric_column_names = train.select_dtypes(include='number').columns

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 20 months
        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)

# Remove rows where target cannot be calculated (last 3 months for each location)
ts_train = ts_train.dropna().reset_index(drop=True)

X_train = ts_train.drop(columns=['target_profitable_3m'])
y_train = ts_train['target_profitable_3m']

print(f"Created features: {ts_train.columns.shape} - 1 target.")
print("Tagert column: target_profitable_3m")
print(f"Final dataset shape: {ts_train.shape}")

ts_train['target_profitable_3m'].value_counts()

/tmp/ipykernel_6014/3536412009.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
/tmp/ipykernel_6014/3536412009.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
/tmp/ipykernel_6014/3536412009.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once us

Created features: (109,) - 1 target.
Tagert column: target_profitable_3m
Final dataset shape: (790702, 109)


target_profitable_3m
0    470137
1    320565
Name: count, dtype: int64

In [9]:
ts_train[ts_train.zipcode_1m==30002] #, ts_train.shape

,target_profitable_3m,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,median_ppsf_2m,median_ppsf_3m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
0,0,125500.0,61870.0,46550.0,217450.0,245000.0,217450.0,63.913043,40.723982,31.813674,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,202681.309539,202421.064584,200773.999557
1,0,153000.0,125500.0,61870.0,189900.0,217450.0,245000.0,81.598080,63.913043,40.723982,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,202998.603897,202681.309539,202421.064584
2,0,165500.0,153000.0,125500.0,154000.0,189900.0,217450.0,81.598080,81.598080,63.913043,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,203781.903446,202998.603897,202681.309539
3,0,140250.0,165500.0,153000.0,145000.0,154000.0,189900.0,78.918228,81.598080,81.598080,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,205012.252913,203781.903446,202998.603897
4,0,140000.0,140250.0,165500.0,103950.0,145000.0,154000.0,78.918228,78.918228,81.598080,...,5408.0,5408.0,5408.0,5408.0,2492.0,2492.0,2492.0,205414.599202,205012.252913,203781.903446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,0,602750.0,555000.0,519000.0,542500.0,540000.0,525000.0,234.281690,230.061350,218.750000,...,5713.0,5713.0,5713.0,5713.0,2728.0,2728.0,2728.0,499412.241077,495606.636012,488436.530983
123,0,568750.0,602750.0,555000.0,455000.0,542500.0,540000.0,232.192450,234.281690,230.061350,...,5713.0,5713.0,5713.0,5713.0,2728.0,2728.0,2728.0,500013.225078,499412.241077,495606.636012
124,0,570000.0,568750.0,602750.0,472642.5,455000.0,542500.0,225.866425,232.192450,234.281690,...,5713.0,5713.0,5713.0,5713.0,2728.0,2728.0,2728.0,498003.211865,500013.225078,499412.241077
125,0,572500.0,570000.0,568750.0,460285.0,472642.5,455000.0,231.490967,225.866425,232.192450,...,5713.0,5713.0,5713.0,5713.0,2728.0,2728.0,2728.0,496887.003144,498003.211865,500013.225078


In [10]:
ts_test = pd.DataFrame()
# Create 3-month profitability target
#ts_test['target_profitable_3m'] = (grouped_test['price'].shift(-3) > 1.02 * test['price']).astype(int)
ts_test['target_profitable_3m'] = (grouped_test['price'].shift(-3) > 1.02 * grouped_test['price'].shift(0)).astype(int)

numeric_column_names = test.select_dtypes(include='number').columns

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 6 months
        ts_test[f'{feature}_{lag}m'] = grouped_test[f'{feature}'].shift(lag)

ts_test = ts_test.dropna().reset_index(drop=True)

X_test = ts_test.drop(columns=['target_profitable_3m'])
y_test = ts_test['target_profitable_3m']

print(f"Created features: {ts_test.columns.shape} - 1 target.")
print("Tagert column: target_profitable_3m")
print(f"Final dataset shape: {ts_test.shape}")

X_train.shape, y_train.shape #, y_train, y_train.value_counts()

/tmp/ipykernel_6014/1004942253.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_test[f'{feature}_{lag}m'] = grouped_test[f'{feature}'].shift(lag)
/tmp/ipykernel_6014/1004942253.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_test[f'{feature}_{lag}m'] = grouped_test[f'{feature}'].shift(lag)
/tmp/ipykernel_6014/1004942253.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

Created features: (109,) - 1 target.
Tagert column: target_profitable_3m
Final dataset shape: (56034, 109)


((790702, 108), (790702,))

In [11]:
#ts_train.columns.values

In [12]:
#ts_train.loc[120:135,['price_1m', 'zipcode_1m']] # Check on the data.

In [13]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
model = SGDClassifier(loss='log_loss')
model.fit(X_train_scaled, y_train)

,loss,'log_loss'
,penalty,'l2'
,alpha,0.0001
,l1_ratio,0.15
,fit_intercept,True
,max_iter,1000
,tol,0.001
,shuffle,True
,verbose,0
,epsilon,0.1
,n_jobs,None


In [22]:
y_pred = model.predict(X_test_scaled)
type(y_pred)

numpy.ndarray

In [16]:
model.score(X_test_scaled, y_test)

0.7101045793625299

In [17]:
precision_score(y_test, y_pred)

0.2541615289765721

In [18]:
y_pred_proba = model.predict_proba(X_test_scaled) # The user should get the probability of the suggestion of the model.
y_pred_proba

array([[0.73300539, 0.26699461],
       [0.72201969, 0.27798031],
       [0.71292546, 0.28707454],
       ...,
       [0.57467835, 0.42532165],
       [0.66399185, 0.33600815],
       [0.71940562, 0.28059438]], shape=(56034, 2))

In [19]:
confusion_matrix(y_test, y_pred)

array([[38141,  4839],
       [11405,  1649]])

In [20]:
y_test.shape

(56034,)

In [21]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.77      0.89      0.82     42980
           1       0.25      0.13      0.17     13054

    accuracy                           0.71     56034
   macro avg       0.51      0.51      0.50     56034
weighted avg       0.65      0.71      0.67     56034



SGD Classifier look on the precision on the test set.
Gradient boosting classifier.

XGBoost classifier.

-> Target might be changed to a regression where we predict the price of the zip code.

-> Convert zipcode into longitude and latitude if possioble. -> Advantage: The model can differenciate the distance between the cities.

Scale with min max scaler as always not.

For final product:

1. Convert zipcodes (delete) into coordinates (add two colums) -> update data frame.  -> check which data set works better.
2. Compare SGDClassifier with Gradient boosting classifier. -> also here with both data sets. -> Decision what to use and may try XGBoost classifier as well (optional).
3. Create another train and test set -> train all but the last month in 2023, test = last month 2023. -> Take care of the dimensions.
4. Create a dataframe with the predictions -> y_pred = model.predict(X_test_scaled) -> one result per zip code. -> After this I can use to start the API.
5. Execute data analytics on it -> where are profitable zipcodes where are not? -> plots, plots, plots!
6. User wants do know in which zipcode a profitable investment is likely. -> Close to a specific zipcode e. g. a new employer.
